In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pickle
import os
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
options = Options()
options.add_argument("--headless")

In [8]:
df = pd.read_csv("../Links.csv")
df = df.head()
for local_df in df.Link.unique():
    for i in local_df.iterrows():
        name = i[1][0]
        link = i[1][1]

AttributeError: 'str' object has no attribute 'iterrows'

In [ ]:
os.system("clear")
with open("/home/amir/github/Amir-personal/facebook-userName-and-password_3.txt", "r") as file:
	usrname, pas = file.read().splitlines()
df = pd.read_csv("../Links.csv")
Successfully_logedin = True
Successfully_logedin_num = 0
while Successfully_logedin:
	Successfully_logedin_num += 1
	if Successfully_logedin_num > 1:
		print(f"Attempt no. {Successfully_logedin_num} to Login")
	try:
		browser = webdriver.Firefox(executable_path="/home/amir/github/working/Facebook_posts_links/geckodriver")
# 		browser = webdriver.Firefox(executable_path = "/home/amir/github/working/Facebook_posts_links/geckodriver", options=options)
		#navigates you to the facebook page.
		browser.get('https://www.facebook.com/')

		#find the username field and enter the email example@yahoo.com.
		time.sleep(np.random.randint(3, 6))
		username = browser.find_elements_by_css_selector("input[name=email]")
		username[0].send_keys(usrname)


		#find the password field and enter the password password.
		time.sleep(np.random.randint(3, 6))
		password = browser.find_elements_by_css_selector("input[name=pass]")
		password[0].send_keys(pas)


		#find the login button and click it.
		time.sleep(np.random.randint(3, 6))
		loginButton = browser.find_elements_by_css_selector("input[type=submit]")
		loginButton[0].click()
		print("Successfully Logged in", current_time())
		Successfully_logedin = False
	except:
		pass
counter = 0

for fb in FB:
	c = 0
	counter += 1
	if not fb in ids_removed_from_facebook:
		complted_url = fb_base_url + fb
		if not fb in all_links:
			print(f"new id added: <{fb_base_url + fb}>")
			all_links[fb] = []
		browser.get(complted_url)
		s = BeautifulSoup(browser.page_source, "lxml")

		##################################################
		# only for groups 
		if fb.startswith("groups"):
			for i in s.select("a"):
				try:
					link = i['href']
					if link.startswith(f"/{fb}permalink/"):
						link = "https://web.facebook.com" + link
						# print("a", end="|")
						if not link in str(all_links):
							# print("b", end="|")
							all_links[fb].append((link, str(now)))
							links_to_open.append(link)
							try:
								browser.get(link)
								# print("c", end="|")
								post = BeautifulSoup(browser.page_source).find("div", {"data-testid" : "post_message"}).text.replace("<br/>", "\n")
								# print("d", end="|")
								if post:
									# print("e", end="|")
									file_name = f"{fb.strip('/').split('/')[1]}.txt"
									# print("f", end="|")
									file = open(file_name, "a+")
									file.write("\n" + "#"*30 + "\n")
									file.write(link + "\n")
									file.write(post + "\n")
									file.close()
									succussfully_extracted += 1
							except:
								errors.append([fb, link])
				except:
					pass
		##################################################	
		else:
			a = s.find("div", {"id" : "timeline_story_column"})
			mmz.append(a)
			try:
				links_ = a.select('a')
				for i in links_:
					link = i['href']
					if link.startswith("https://web.facebook.com/"):
						if not link in str(all_links):
							if "/posts/" in link: # failing to capture like this link (https://web.facebook.com/permalink.php?story_fbid=129882195152447&id=100043920022318)
								if not "?comment_id=" in link:
									all_links[fb].append((link, str(now)))
									links_to_open.append(link)
									c += 1
									try:
										browser.get(link)
									except:
										errors.append([fb, link])
										continue
									soup = BeautifulSoup(browser.page_source, "lxml")
									try:
										aa = soup.find("div", {"class" : "_5wj-"}).text
										if len(aa) > 0:
											file_name = f"{fb}.txt"
											file = open(file_name, "a+")
											file.write("\n" + "#"*30 + "\n")
											file.write(link + "\n")
											file.write(aa + "\n")
											succussfully_extracted += 1
											file.close()
										else:
											errors.append([fb, link])
									except:
										errors.append([fb, link])
										pass

			except:
				continue
	perc = counter/len(FB)*100
	print("{:3} {} %  || {:2} of {}  ||  ".format(int(perc), " ", counter, len(FB)),
						 current_time(),
						 f" ||  {c} links in {fb}")

browser.close()

if not links_to_open:
	if "check" in os.listdir():
		import shutil
		shutil.rmtree("check", ignore_errors=False)
	os.mkdir("check")
	os.chdir("check/")
	for e, i in enumerate(mmz):
		with open(str(e) + ".txt", "w") as file:
			file.write(str(i))
	from termcolor import colored
	print(colored("\n\n<check> folder created, you can check there why you not get any link\n\n", 'red'))


links_qty_after_addition = sum([len(all_links[i]) for i in all_links])

if succussfully_extracted:
	print("New links Qty: ", links_qty_after_addition - stored_links_qty)
	print("\n\nsuccussfully extracted: ", succussfully_extracted)
	with open("All_FB_links_names_corrected.pkl", "wb") as file:
		pickle.dump(all_links, file)
	for i in links_to_open:
		os.popen("firefox " + i)

else:
	with open("errors.txt", "w") as file:
		for error in errors:
			file.write(error[0] + ":  " + error[1] + "\n")
	print(f"""\n\nNo link was extracted succussfully, you can need bug fixing we saved all the scrapped data at: 
		/home/amir/github/working/Facebook_posts_links/check/ \n\n""")


try:
	os.remove("geckodriver.log")
except:
	pass

os.system("ipython3 /home/amir/github/working/Facebook_posts_links/last_post.py")